In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import pandas as pd
import numpy as np

In [2]:
url_base = 'https://www.chicagomag.com/'
url_sub =  'Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub
url

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'

In [4]:
# 요청 후 응답(코드 추출)
# HTTPError: HTTP Error 403: Forbidden(부정접근 차단 헤더에 agent를 추가)
# 헤더 추가시(urllib 패키지인 경우) Request() 객체 사용 후 open
req = Request(url,headers={'User-Agent':'Mozilla/5.0'})
html = urlopen(req)

In [5]:
soup = BeautifulSoup(html,'html.parser')
soup

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<script src="https://cmp.osano.com/16A1AnRt2Fn8i1unj/f15ebf08-7008-40fe-9af3-db96dc3e8266/osano.js"></script>
<title>The 50 Best Sandwiches in Chicago – Chicago Magazine</title>
<style type="text/css">
					.heateor_sss_button_instagram span.heateor_sss_svg,a.heateor_sss_instagram span.heateor_sss_svg{background:radial-gradient(circle at 30% 107%,#fdf497 0,#fdf497 5%,#fd5949 45%,#d6249f 60%,#285aeb 90%)}
						div.heateor_sss_horizontal_sharing a.heateor_sss_button_instagram span{background:#000!important;}div.heateor_sss_standard_follow_icons_container a.heateor_sss_button_instagram span{background:#000;}
										.heateor_sss_horizontal_sharing .heateor_sss_svg,.heateor_sss_standard_follow_icons_container .heateor_sss_svg{
							background-color: #000!important;
				background: #000!important;
							color: #f

In [8]:
# 랭킹되어 있는 샌드위치 가게 목록 찾아오기
sand_all = soup.find_all('div',{'class':'sammy'})
# 다른표현
soup.find_all('div','sammy')

[<div class="sammy" style="position: relative;">
 <div class="sammyRank">1</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
 Old Oak Tap<br/>
 <em>Read more</em> </a></div>
 </div>,
 <div class="sammy" style="position: relative;">
 <div class="sammyRank">2</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/"><b>Fried Bologna</b><br/>
 Au Cheval<br/>
 <em>Read more</em> </a></div>
 </div>,
 <div class="sammy" style="position: relative;">
 <div class="sammyRank">3</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/"><b>Woodland Mushroom</b><br/>
 Xoco<br/>
 <em>Read more</em> </a></div>
 </div>,
 <div class="sammy" style="position: relative;">
 <div class="sammyRank">4</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-i

In [7]:
len(sand_all)

50

In [9]:
sand_all[0]

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>

In [12]:
sand_all[0].find(class_='sammyRank').text
sand_all[0].find(class_='sammyRank').get_text()

'1'

In [13]:
sand_all[0].find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [16]:
temp = sand_all[0].find(class_='sammyListing').get_text()

In [18]:
temp.split('\n')

['BLT', 'Old Oak Tap', 'Read more ']

In [19]:
# 정규식 패키지
import re

In [20]:
re.split(('\n|\r\n'),temp)

['BLT', 'Old Oak Tap', 'Read more ']

In [21]:
url_base

'https://www.chicagomag.com/'

In [22]:
url_base+sand_all[0].find('a')['href']

'https://www.chicagomag.com//Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [23]:
# url 구성 함수
from urllib.parse import urljoin

In [24]:
urljoin(url_base,sand_all[0].find('a')['href'])

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

### ---------코드 결합----------------
- 전체 50개 상점 정보를 df에 저장

In [33]:
rank = []
main_menu= []
cafe_name = []
url_link = []

In [26]:
url_base = 'https://www.chicagomag.com/'
url_sub =  'Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub
# url
import ssl
context = ssl._create_unverified_context()
# html = urlopen(url) #403번 에러 발생(부정접근으로 차단-알려진 agent를 헤더에 추가해서 전달하면 됨)
# 헤더를 추가하려면 일반적으로 Request(get_url,headers={key:value})
req = Request(url,headers={'User-Agent':'Mozilla/5.0'})
html=urlopen(req,context=context)

soup = BeautifulSoup(html,'html.parser')
soup_list = soup.find_all('div','sammy')

In [34]:
for item in soup_list : 
    rank.append(item.find(class_='sammyRank').get_text())
    tmp = item.find(class_='sammyListing').get_text()
    main_menu.append(re.split(('\n|\r\n'),tmp)[0])
    cafe_name.append(re.split(('\n|\r\n'),tmp)[1])
    url_link.append(urljoin(url_base,item.find('a')['href']))                     

In [35]:
len(rank),len(main_menu),len(cafe_name),len(url_link)

(50, 50, 50, 50)

In [36]:
# df 로 생성 후 csv 저장
data = {'Rank':rank,'Cafe':cafe_name, 'Menu':main_menu, 'URL':url_link}

df=pd.DataFrame(data)

In [37]:
df.tail()

,Rank,Cafe,Menu,URL
45,46,Chickpea,Kufta,https://www.chicagomag.com/Chicago-Magazine/No...
46,47,The Goddess and Grocer,Debbie’s Egg Salad,https://www.chicagomag.com/Chicago-Magazine/No...
47,48,Zenwich,Beef Curry,https://www.chicagomag.com/Chicago-Magazine/No...
48,49,Toni Patisserie,Le Végétarien,https://www.chicagomag.com/Chicago-Magazine/No...
49,50,Phoebe’s Bakery,The Gatsby,https://www.chicagomag.com/Chicago-Magazine/No...


In [38]:
df.to_csv('./crawl_data/시카고샌드위치가게.csv',sep=',',encoding='utf-8')

#### 주소 크롤링 후 지도 시각화

In [2]:
df = pd.read_csv('./crawl_data/시카고샌드위치가게.csv',index_col=0)

In [3]:
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


In [4]:
df['URL'][0]

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [5]:
import ssl
context = ssl._create_unverified_context()
req = Request(df['URL'][0],headers={'User-Agent':'Mozilla/5.0'})
html = urlopen(req,context=context)
soup_tmp = BeautifulSoup(html,'html.parser')


In [7]:
tmp_str = soup_tmp.find('p','addy').get_text()

In [15]:
tmp_str.split()#[0]#[:-1]

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [13]:
' '.join(tmp_str.split()[1:-2])

'2109 W. Chicago Ave.,'

In [17]:
df.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
           dtype='int64')

In [16]:
# 전체 데이터 추출
price = []
address = []

In [18]:
from tqdm import tqdm_notebook
for i in tqdm_notebook(df.index) :
     
    html = urlopen(req,context=context)
    soup_tmp = BeautifulSoup(html,'html.parser')
    tmp_str = soup_tmp.find('p','addy').get_text()
    price.append(tmp_str.split()[0][:-1])
    address.append(' '.join(tmp_str.split()[1:-2]))

C:\Users\TECH4-29\AppData\Local\Temp\ipykernel_7688\2274314564.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(df.index) :


  0%|          | 0/50 [00:00<?, ?it/s]

In [19]:
len(price),len(address)

(50, 50)

In [20]:
df['price']=price
df['address']=address

In [21]:
df.head()

,Rank,Cafe,Menu,URL,price,address
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


In [22]:
df = df.loc[:,['Rank','Cafe','Menu','price','address']]
df.head()

,Rank,Cafe,Menu,price,address
0,1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,"
1,2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,"
2,3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,"
3,4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,"
4,5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,"


In [23]:
df.set_index('Rank',inplace=True)

In [24]:
df.tail()

,Cafe,Menu,price,address
Rank,,,,
46,Chickpea,Kufta,$8,"2018 W. Chicago Ave.,"
47,The Goddess and Grocer,Debbie’s Egg Salad,$6.50,"25 E. Delaware Pl.,"
48,Zenwich,Beef Curry,$7.50,"416 N. York St., Elmhurst,"
49,Toni Patisserie,Le Végétarien,$8.75,"65 E. Washington St.,"
50,Phoebe’s Bakery,The Gatsby,$6.85,"3351 N. Broadway,"


In [25]:
## 시카고샌드위치_주소.csv
df.to_csv('./crawl_data/시카고샌드위치_주소.csv',sep=',',encoding='utf-8')

### 수집된 주소를 이용해서 각 상점의 위경도 찾기
- 구글맵스를 이용해서 기본 map 가져오고 위경도 찾기

In [ ]:
!pip install googlemaps

In [26]:
import googlemaps
import folium
import pandas as pd
import numpy as np

In [29]:
# 구글 클라이언트 등록
gmapsKey='본인키'
gmaps = googlemaps.Client(key=gmapsKey)

In [35]:
# googlemaps.client.Client객체의 geocode(타겟)
gmaps.geocode('서울')[0]['geometry']['location']['lat']

37.5518911

In [41]:
target = df['address'][1]+','+'Chicago'
gmaps.geocode(target)[0]['geometry']['location']['lat']

41.8955577

In [43]:
# 50개 위경도
lat =[]
lng =[]

In [44]:
for n in tqdm_notebook(df.index) :
    target = df['address'][n]+','+'Chicago'
    g_info = gmaps.geocode(target)
    g_lo = g_info[0].get("geometry")['location']
    lat.append(g_lo['lat'])
    lng.append(g_lo['lng'])

C:\Users\TECH4-29\AppData\Local\Temp\ipykernel_7688\1965740440.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(df.index) :


  0%|          | 0/50 [00:00<?, ?it/s]

In [45]:
len(lat),len(lng)

(50, 50)

In [46]:
df['lat']=lat
df['lng']=lng

In [47]:
df.head()

,Cafe,Menu,price,address,lat,lng
Rank,,,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,",41.895558,-87.679967
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,",41.884639,-87.647590
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,",41.890523,-87.630783
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,",42.058322,-87.683748
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,",41.886604,-87.648536


### 지도 시각화
- folium map : import folium
- map = folium.Map(location=[위,경도], zoom_start='초기확대정도')
### 마커
- folium.Marker([위도,경도],popup='마커출력글자').add_to(map지도객체)

In [48]:
# 수집한 위경도 평균을 구해서 지도의 중심위치로 설정
lat_c = df['lat'].mean()
lng_c = df['lng'].mean()

In [49]:
map = folium.Map(location=[lat_c,lng_c],zoom_start=11)
folium.Marker([lat_c,lng_c],popup='Center').add_to(map)

In [50]:
map

- 전체 cafe의 위치에 Marker 표시

In [51]:
map_fin = folium.Map(location=[lat_c,lng_c],zoom_start=11)

In [52]:
for n in df.index :
    folium.Marker([df['lat'][n],df['lng'][n]],
                  popup = df['Cafe'][n]).add_to(map_fin)

In [53]:
map_fin

In [54]:
# 지도 저장
map_fin.save('./crawl_data/시카고_카페.html')

In [55]:
# df 저장
df.to_csv('./crawl_data/시카고_위경도.csv')